## SLT Trainer

#### DataLoader

In [1]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator

# Usage example
roi_folder = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\processed_setfile"
eeg_folder = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\setfile"
group_file = "./Dataloader/subject_groups.json"

# Create dataset
train_dataset = EEGROIDataset(roi_folde0r, eeg_folder, group_file, "train_dataset")
print(f"Total dataset size: {len(train_dataset)}")
# Create dataset
test_dataset = EEGROIDataset(roi_folder, eeg_folder, group_file, "eval")
print(f"Total dataset size: {len(test_dataset)}")

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '314', '20']
ROI Times: 302.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\0_ICA_DLtrain.fdt
Reading 0 ... 77496  =      0.000 ...   302.719 secs...
Total Nan of the subject: 0
ROI Times: 308.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\1_ICA_DLtrain.fdt
Reading 0 ... 78878  =      0.000 ...   308.117 secs...
Total Nan of the subject: 0
ROI Times: 300.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\2_ICA_DLtrain.fdt
Reading 0 ... 77004  =      0.000 ...   300.797 secs...
Total Nan of the subject: 0
ROI Times: 254.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\3_ICA_DLtrain.fdt
Reading 0 ... 66080  =      0.000 ...   258.125 secs...


G:\共用雲端硬碟\CNElab_陳昱祺\multi-modal\Dataloader\SLT_dataloader.py:56: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_data = mne.io.read_raw_eeglab(eeg_path, preload=True).get_data()


Total Nan of the subject: 0
ROI Times: 304.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\4_ICA_DLtrain.fdt
Reading 0 ... 78059  =      0.000 ...   304.918 secs...
Total Nan of the subject: 0
ROI Times: 302.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\5_ICA_DLtrain.fdt
Reading 0 ... 77393  =      0.000 ...   302.316 secs...
Total Nan of the subject: 0
ROI Times: 316.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\6_ICA_DLtrain.fdt
Reading 0 ... 81336  =      0.000 ...   317.719 secs...
Total Nan of the subject: 0
ROI Times: 234.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\7_ICA_DLtrain.fdt
Reading 0 ... 61092  =      0.000 ...   238.641 secs...


G:\共用雲端硬碟\CNElab_陳昱祺\multi-modal\Dataloader\SLT_dataloader.py:56: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_data = mne.io.read_raw_eeglab(eeg_path, preload=True).get_data()


Total Nan of the subject: 0
ROI Times: 238.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\8_ICA_DLtrain.fdt
Reading 0 ... 62045  =      0.000 ...   242.363 secs...


...

Total Nan of the subject: 0
Total dataset size: 3069
['49', '345', '364', '422', '90']
ROI Times: 228.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\49_ICA_DLtrain.fdt
Reading 0 ... 59760  =      0.000 ...   233.438 secs...


G:\共用雲端硬碟\CNElab_陳昱祺\multi-modal\Dataloader\SLT_dataloader.py:56: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_data = mne.io.read_raw_eeglab(eeg_path, preload=True).get_data()


Total Nan of the subject: 0
ROI Times: 312.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\345_ICA_DLtrain.fdt
Reading 0 ... 80220  =      0.000 ...   313.359 secs...
Total Nan of the subject: 0
ROI Times: 300.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\364_ICA_DLtrain.fdt
Reading 0 ... 77168  =      0.000 ...   301.438 secs...
Total Nan of the subject: 0
ROI Times: 300.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\422_ICA_DLtrain.fdt
Reading 0 ... 76799  =      0.000 ...   299.996 secs...
Total Nan of the subject: 0
ROI Times: 228.0
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\90_ICA_DLtrain.fdt
Reading 0 ... 59464  =      0.000 ...   232.281 secs...


G:\共用雲端硬碟\CNElab_陳昱祺\multi-modal\Dataloader\SLT_dataloader.py:56: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_data = mne.io.read_raw_eeglab(eeg_path, preload=True).get_data()


Total Nan of the subject: 0
Total dataset size: 684


#### Model

In [2]:
import sys
sys.path.insert(0, './FirstMultiModel/EEGART')
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from SLT_dataloader import SignalDataCollator

from tf_config import SLTConfig
from tf_model import SLTModel
from datetime import datetime

# 获取当前日期时间，并格式化为 YYYYMMDD
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")

# 替换路径中的日期
output_dir = f"./results/{current_date}_model_init_LR_05"

# 假設你的設備是 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# slt_config = SLTConfig(src_channel_size=30, tgt_channel_size=3*5003, N=2, d_model = 1024)
# slt_config.save_pretrained("test_slt_confit")

# slt_model = SLTModel(slt_config)
# slt_model = slt_model.to(device)

    
def compute_metrics(eval_preds, batch_size=64):
    predictions, targets = eval_preds
    
    loss_fct = nn.MSELoss()
    num_samples = predictions.shape[0]
    mse_sum = 0.0
    num_batches = 0

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        logits_batch = predictions[start_idx:end_idx]
        labels_batch = targets[start_idx:end_idx]
        logits = torch.tensor(logits_batch, dtype=torch.float32)
        labels = torch.tensor(labels_batch, dtype=torch.float32)
        # Compute the z-scores for the batch
        logits_mean = torch.mean(logits, dim=0, keepdim=True)
        logits_std = torch.std(logits, dim=0, keepdim=True)
        logits_norm = (logits - logits_mean) / (logits_std + 1e-10)
        
        labels_mean = torch.mean(labels, dim=0, keepdim=True)
        labels_std = torch.std(labels,  dim=0, keepdim=True)
        labels_norm = (labels - labels_mean) / (labels_std + 1e-10)

        if torch.isnan(logits_norm).any() or torch.isinf(logits_norm).any():
            print("logits_norm contains nan or inf")
        if torch.isnan(labels_norm).any() or torch.isinf(labels_norm).any():
            print("labels_norm contains nan or inf")
        
        # Compute batch MSE
        mse_sum += loss_fct(logits_norm, labels_norm).item()
        num_batches += 1

    # Return the average MSE
    avg_mse = mse_sum / num_batches
    
    del logits, labels, logits_norm, labels_norm
    torch.cuda.empty_cache()
    
    return {"mse": avg_mse}

# 包裝自定義的初始化方法
def huggingface_model_init():
    slt_config = SLTConfig(src_channel_size=30, tgt_channel_size=3*5003, N=2, d_model = 1024)
    slt_config.save_pretrained("test_slt_confit")
    slt_model = SLTModel(slt_config)
    slt_model = slt_model.to(device)
    for p in slt_model.parameters():
        if p.dim() > 1:
            torch.nn.init.xavier_uniform_(p)
    return slt_model



training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="steps",
    eval_steps=48,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_accumulation_steps=5,
    num_train_epochs=30,
    weight_decay=0.01,
    learning_rate = 0.01,
    lr_scheduler_type = "constant",
    # fp16=True,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
)

# 初始化模型和 Trainer
trainer = Trainer(
    # model=slt_model,
    model_init=huggingface_model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=SignalDataCollator(),
    compute_metrics=compute_metrics,
)

# 开始训练
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: richielp700 (richielp700-national-tsing-hua-university). Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Mse
48,1.939900,1.936036,1.967581
96,1.937600,1.936847,1.970164
144,1.939900,1.933460,1.965250
192,1.939700,1.938017,1.969725
240,1.925400,1.937559,1.969669
288,1.946500,1.934961,1.967475
336,1.927400,1.932846,1.964924
384,1.943300,1.935062,1.967464
432,1.935700,1.933609,1.965299
480,1.940100,1.934230,1.967149


TrainOutput(global_step=2880, training_loss=1.9375227669874826, metrics={'train_runtime': 47260.378, 'train_samples_per_second': 1.948, 'train_steps_per_second': 0.061, 'total_flos': 0.0, 'train_loss': 1.9375227669874826, 'epoch': 30.0})